# A minimal ALF run

In this bare-bones example we use the [pyALF](https://git.physik.uni-wuerzburg.de/ALF/pyALF/-/tree/master/) interface to run the canonical Hubbard model on a default configuration: a $6\times6$ square grid, with interaction strength $U=4$ and inverse temperature $\beta = 5$.  

Bellow we go through the steps for performing the simulation and outputting observables.

---

**1.** Import `ALF_source` and `Simulation` classes from the `py_alf` python module, which provide the interface with ALF:

In [1]:
from py_alf import ALF_source, Simulation  # Interface with ALF

**2.** Create an instance of `ALF_source`, downloading the ALF source code from the [ALF repository](https://git.physik.uni-wuerzburg.de/ALF/ALF/-/tree/master/), if `alf_dir` does not exist. Gets alf_dir from environment variable `$ALF_DIR`, or defaults to "./ALF", if not present:

In [2]:
alf_src = ALF_source()

**3.** Create an instance of `Simulation`, overwriting default parameters as desired:

In [3]:
sim = Simulation(
    alf_src,
    "Hubbard",                    # Name of Hamiltonian
    {                             # Dictionary overwriting default parameters
        "Lattice_type": "Square"
    },
    machine='GNU'  # Change to "intel", or "PGI" if gfortran is not installed
)

**4.** Compile ALF. The first time it will also download and compile HDF5, which could take $\sim15$ minutes.

In [4]:
sim.compile()

Compiling ALF... 
Done.


**5.** Perform the simulation as specified in `sim`:

In [5]:
sim.run()

Prepare directory "/home/stafusa/ALF/pyALF/Notebooks/ALF_data/Hubbard_Square" for Monte Carlo run.
Resuming previous run.
Run /home/stafusa/ALF/pyALF/Notebooks/ALF/Prog/ALF.out


**6.** Perform some simple analysis:

In [6]:
sim.analysis()

### Analyzing /home/stafusa/ALF/pyALF/Notebooks/ALF_data/Hubbard_Square ###
/home/stafusa/ALF/pyALF/Notebooks
Scalar observables:
Ener_scal
Kin_scal
Part_scal
Pot_scal
Histogram observables:
Equal time observables:
Den_eq
Green_eq
SpinT_eq
SpinXY_eq
SpinZ_eq
Time displaced observables:
Den_tau
Green_tau
SpinT_tau
SpinXY_tau
SpinZ_tau


**7.** Read analysis results into a Pandas Dataframe with one row per simulation, containing parameters and observables:

In [7]:
obs = sim.get_obs()

/home/stafusa/ALF/pyALF/Notebooks/ALF_data/Hubbard_Square


In [8]:
obs

,ham_t,ham_chem,ham_u,ham_t2,ham_u2,ham_tperp,mz,continuous,model,lattice_type,...,SpinXY_tauK,SpinXY_tauK_err,SpinXY_tauR,SpinXY_tauR_err,SpinXY_tau_lattice,SpinZ_tauK,SpinZ_tauK_err,SpinZ_tauR,SpinZ_tauR_err,SpinZ_tau_lattice
/home/stafusa/ALF/pyALF/Notebooks/ALF_data/Hubbard_Square,1.0,0.0,4.0,1.0,4.0,1.0,1,0,b'Hubbard',b'Square',...,"[[0.8844797151537013, 0.6832868201255057, 0.60...","[[0.11619601518306225, 0.11389830399256165, 0....","[[0.09997767446757408, -0.08263155523732026, 0...","[[0.029679282848879707, 0.013927995255301738, ...","{'L1': [6.0, 0.0], 'L2': [0.0, 6.0], 'a1': [1....","[[0.8188570095635151, 0.6190690150139192, 0.57...","[[0.04298020361350671, 0.03379901002616846, 0....","[[0.08650153433812775, -0.09479509450016295, 0...","[[0.018104357313031, 0.019927569547298896, 0.0...","{'L1': [6.0, 0.0], 'L2': [0.0, 6.0], 'a1': [1...."


$\bullet$ The internal energy of the system (and its error) are accessed by:

In [9]:
obs.iloc[0][['Ener_scal0', 'Ener_scal0_err', 'Ener_scal_sign', 'Ener_scal_sign_err']]

Ener_scal0            -29.8542
Ener_scal0_err        0.091859
Ener_scal_sign             1.0
Ener_scal_sign_err         0.0
Name: /home/stafusa/ALF/pyALF/Notebooks/ALF_data/Hubbard_Square, dtype: object

$\bullet$ The simulation can be resumed by calling `sim.run()` again, increasing the precision of results:

In [10]:
sim.run()
sim.analysis()
obs2 = sim.get_obs()
obs2.iloc[0][['Ener_scal0', 'Ener_scal0_err', 'Ener_scal_sign', 'Ener_scal_sign_err']]

Prepare directory "/home/stafusa/ALF/pyALF/Notebooks/ALF_data/Hubbard_Square" for Monte Carlo run.
Resuming previous run.
Run /home/stafusa/ALF/pyALF/Notebooks/ALF/Prog/ALF.out
### Analyzing /home/stafusa/ALF/pyALF/Notebooks/ALF_data/Hubbard_Square ###
/home/stafusa/ALF/pyALF/Notebooks
Scalar observables:
Ener_scal
Kin_scal
Part_scal
Pot_scal
Histogram observables:
Equal time observables:
Den_eq
Green_eq
SpinT_eq
SpinXY_eq
SpinZ_eq
Time displaced observables:
Den_tau
Green_tau
SpinT_tau
SpinXY_tau
SpinZ_tau
/home/stafusa/ALF/pyALF/Notebooks/ALF_data/Hubbard_Square


Ener_scal0           -29.929472
Ener_scal0_err         0.079442
Ener_scal_sign              1.0
Ener_scal_sign_err          0.0
Name: /home/stafusa/ALF/pyALF/Notebooks/ALF_data/Hubbard_Square, dtype: object

In [11]:
print("\nRunning again reduced the error from\n", obs.iloc[0][['Ener_scal0_err']], "\nto\n", obs2.iloc[0][['Ener_scal0_err']])


Running again reduced the error from
 Ener_scal0_err    0.091859
Name: /home/stafusa/ALF/pyALF/Notebooks/ALF_data/Hubbard_Square, dtype: object 
to
 Ener_scal0_err    0.079442
Name: /home/stafusa/ALF/pyALF/Notebooks/ALF_data/Hubbard_Square, dtype: object


**Note**: To run a fresh simulation - instead of performing a refinement over previous run(s) - the Monte Carlo run directory should be deleted before rerunning.

---
## Exercises
1. Check info/help commands such as `help(alf_src)`, `alf_src.get_ham_names()`, and `alf_src.get_default_params('Hubbard')`.
2. Rerun once again and check the new improvement in precision.
3. Look at a few other observables (`sim.analysis()` outputs the names of those available).
4. Change the lattice size by adding, e.g., `\"L1\": 4,` and `\"L2\": 1,` to the simulation parameters definitions of `sim` (step 2).